<a href="https://colab.research.google.com/github/erickummelstedt/Spiced-Academy-Data-Science-Projects/blob/master/01_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an initial machine learning workflow

Welcome to Exercise 1! After you successfuly completed the Introduction to Python last week (or if you were already familiar with it), we are making use of this knowledge and start building an initial machine learning (ML) workflow. All of the steps will be explored in more detail further in the course, so you don't need to understand everything, the goal of this exercise is to give you a high-level overview of the different stages of a ML workflow.

## Get and curate the data

To achieve this, we will try to predict the wavelength of the π-π* absorption of the E-isomer of azobenzenes. E and Z isomers refer to the configuration of the N=N double bond in the molecule, and the wavelength corresponds to the ligth $h\nu$ absorbed by the azobenzene when switching from the E-isomer to the Z-isomer.

<img src="https://drive.google.com/uc?export=view&id=1rTNH0mt5Aecw2LMw_zgmgW10292ElBMw" width="400"/>

The dataset originally stems from a publication from [Griffiths et al.](https://doi.org/10.1039/D2SC04306H). The one we use has been slightly modified by your teaching team, for example adding molecular properties (also called "descriptors" or "features") of our molecules of interest. Let's go ahead and download our dataset.

In [3]:
df = !wget "https://gitlab.ethz.ch/schmiste/digital_chemistry_fs25/-/raw/main/Exercise1/Ex1_photoswitches.csv"

After we have the dataset, we need to load it. Since it is a csv file, we will load it into a Pandas DataFrame.

In [8]:
import pandas as pd

dataset = pd.read_csv("https://gitlab.ethz.ch/schmiste/digital_chemistry_fs25/-/raw/main/Exercise1/Ex1_photoswitches.csv")# TODO: Complete the command to get the DataFrame from the csv file

# Print the first 5 rows of the dataset
dataset.head()

,SMILES,E isomer pi-pi* wavelength in nm,NumAromaticRings,NumHBD,NumHBA,NumHeteroatoms,NumRotatableBonds,NumAromaticHeterocycles,MolecularWeight
0,C[N]1N=NC(=N1)N=NC2=CC=CC=C2,310.0,2,0,6,6,2,1,188.081044
1,C[N]1C=NC(=N1)N=NC2=CC=CC=C2,310.0,2,0,5,5,2,1,187.085795
2,C[N]1C=CC(=N1)N=NC2=CC=CC=C2,320.0,2,0,4,4,2,1,186.090546
3,C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2,325.0,2,0,4,4,2,1,200.106196
4,C[N]1C=C(C=N1)N=NC2=CC=CC=C2,328.0,2,0,4,4,2,1,186.090546


You should see a dataset which consists of the molecules encoded in SMILES (a string representation of a molecule that you will encounter many times during this course), the absorption wavelength of the π-π* transition of the E-isomer of the molecules, as well as seven different molecular properties that we will use for our prediction.

Unfortunately, it turns out that, even though we calculated the properties for all the molecules in our dataset, we do not have experimental absorption wavelengths for all of them (their value in the DataFrame is "NaNs" (= not a number)). In a first step, let's find out how many molecules we have in total and how many of them do not have a recorded absorption wavelength. Then let's curate the dataset by removing these rows.

In [9]:
#TODO: Print the number of molecules in the original dataset
print(len(dataset))

# Find the number of NaNs in the column E isomer pi-pi* wavelength in nm
number_nans = dataset["E isomer pi-pi* wavelength in nm"].isna().sum()
print(f"Number of molecules with no recorded absorption: {number_nans}")

# Get rid of all rows that contain a NaN value
dataset = dataset.dropna()

#TODO: Print the number of molecules in the curated dataset
print(len(dataset))

405
Number of molecules with no recorded absorption: 13
392


In order to do machine learning, we need to obtain a representation of our molecules as well as a target value to predict (the wavelength in our case). From the DataFrame, extract the representation (the descriptors written in `features_list`) and the target value.

In [17]:
features_list = ['NumAromaticRings', 'NumHBD', 'NumHBA', 'NumHeteroatoms', 'NumRotatableBonds', 'NumAromaticHeterocycles', 'MolecularWeight']

# Extract input (X) and target values (y)
# TODO: Fill in the corresponding columns names in the `[]` below
X = dataset[features_list].to_numpy()
y = dataset[['E isomer pi-pi* wavelength in nm']].to_numpy()

print(X.shape, y.shape)

(392, 7) (392, 1)


## Build predictive models

Great! Now we get to the machine learning part! At first, we are going to fit a linear model to our descriptors to predict the wavelength. With such a linear equation, coefficients can be analysed, which gives us an idea of how important each of the descriptors are for the prediction of the wavelength.
Afterwards, we will switch to a non-linear model, namely random forest. Non-linear models often allow to fit more complex functions that are, in fact, not linear. Again for now, it's about getting a first feeling for training ML models, but you will see them in more details further in the course (🔗 Lecture 3).

To get us started, first we load some useful functions from the scikit-learn library, an important Python library for ML.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import matplotlib.pyplot as plt

### Train and test split

Before we train a model, we first have to split the data into a training set and a testing set (🔗 Lecture 4). Put simply, the training data is used for training the model, i.e. fitting the coefficients of our linear regression in this case. The test set is then used for assessing the generalization performance of our model, meaning how well the model does prediction on unseen molecules that it was not trained on. A common strategy for splitting a dataset is to have 80% of the data points for training and keeping 20% for testing, a strategy which we will apply here.

In [20]:
# Split the dataset into 0.8/0.2 train/test sets. We set the random seed (arbitrarily to 1) for reproducibility.
# TODO: enter the test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Features standardisation

Since having descriptors with different scale would make the linear model focus too much on features with large values and ignore the small ones, we first have to standardise our descriptors, i.e. transform them in such a way that each descriptor's distribution is centered around 0 with a standard deviation of 1 (🔗 Lecture 4). Like this, all descriptors will have the same order of magnitude, which enables their coefficients to be comparable.

It is important that you fit the scaler (the object that performs the standardisation) only on the train set and not the test set, as we need to let the test data really remain unseen by the model (otherwise, we have a problem called "data leakage", more about this in 🔗 Lecture 4).

In [ ]:
# Standardise the train and test sets
scaler = StandardScaler()
 = scaler.fit_transform() # TODO: Complete the command to fit and transform the correct dataset
 = scaler.transform() # TODO: Complete the command to transform the correct dataset

### Model training and testing

Now that we have a standardised representation for the molecules, we are ready to perform the linear regression. At first, let's fit the regression model on the train set, and then make predictions on the test set.

In [ ]:
# Train the linear regression model and then perform predictions on the test set
lin_reg = LinearRegression()
lin_reg.fit(X= , y= ) # TODO: enter the correct representation matrix and target vector for training the model
y_pred_lin = lin_reg.predict(X=) # TODO: enter the correct matrix for testing the model

You fitted the first model of the course! We will look at its predictive performance a bit below. Before, let's apply the random forest. We will discuss later in the course how this method works, but for now, fit it to the train set and perform your predictions on the test set similarly as previously.

In [ ]:
# Train the random forest regression model and then perform predictions on the test set
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
# TODO: Analogously to the linear regression, fit the random forest model and predict on the test set. Use the non-standardised matrices for train and test representations

### Model evaluation

Given that we now have our predictions both from the linear and random forest regressions, we can compare how well these models perform on predicting the absorption wavelength of unseen examples (from our test set). To compare them, compute the $R^2$ score, mean absolute error (MAE) and root mean squared error (RMSE) between the true values (`y_test`) and the predicted values (for both models separately). We will expand on the significance of these metrics later in the course (🔗 Lecture 3) - but in short, the $R^2$ score indicates the correlation between true and the predicted values (higher is better), while the MAE and RMSE indicate the deviation between the true and the predicted values (lower is better), with RMSE being more sensitive to outliers.

At the end, we print out these values to compare how well these models perform.

In [ ]:
# Calculate the r2 score, MAE and MSE between the predicted and the true values in the test set for the two models
r2_lin = r2_score(y_test, y_pred_lin)
mae_lin = mean_absolute_error(y_test, y_pred_lin)
rmse_lin = root_mean_squared_error(y_test, y_pred_lin)
r2_rf = # TODO: Complete analogously to the linear regression
mae_rf = # TODO: Complete analogously to the linear regression
rmse_rf = # TODO: Complete analogously to the linear regression

print(f"Linear Regression R^2: {r2_lin:.3f}, MAE: {mae_lin:.2f} nm, RMSE: {rmse_lin:.2f} nm")
print(f"Random Forest R^2: {r2_rf:.3f}, MAE: {mae_rf:.2f} nm, RMSE: {rmse_rf:.2f} nm")

In addition to calculating various metrics to compare the performance of our two models, we can do a visual inspection. For this, we do a parity plot, i.e. a plot between the true and the predicted values for both models and see how well they correlate. This way, we can also identify whether the model performs better or worse on certain specific points.

In [ ]:
# Create a parity plot
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_lin, label="Linear regression", alpha=0.7)
# TODO: Add the points from the random forest prediction analogously to the line above
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "k--")
plt.legend()
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.title("Parity plot: linear regression vs. random forest")

The model comparison already gives some indication which model performs better on the predictions of unseen cases. While you have just trained (relatively) simple models, your teaching team has trained a neural network on the same dataset. We have trained it with exactly the same train/test split, to ensure that we get an accurate estimate for the model performance.

Instead of descriptors to represent the molecules, we take the Morgan fingerprints of the SMILES as an input, on which we trained a neural network with two hidden layers. Of course, all of these concepts will come up later in the course in more details (🔗 Lectures 2 and 7 more precisely). In this section, we only want to compare the performance of the neural network with the random forest and linear models.

For this, we first download the necessary files (Neural_Network.pt is the saved model, Predict_NN.py is a python file that we use to make our prediction and that handles stuff in the background) and import the required libraries.

In [ ]:
!wget "https://gitlab.ethz.ch/schmiste/digital_chemistry_fs25/-/raw/main/Exercise1/Neural_Network.pt"
!wget "https://gitlab.ethz.ch/schmiste/digital_chemistry_fs25/-/raw/main/Exercise1/Predict_NN.py"

In [ ]:
!pip install torch
!pip install pytorch-lightning
!pip install rdkit

In [ ]:
from Predict_NN import make_predictions

Since your TA team wrote a python function (`make_predictions`) to handle everything for the neural network predictions in the background, you can simple call this function. For arguments, it takes the path to the neural network model file and the Pandas DataFrame. Then, calculate the same metrics as above for the neural network and create a parity plot that includes the neural network.

In [ ]:
# Make the predictions with the neural network
y_pred_NN = make_predictions(model_path=, dataset= ) # TODO: Enter the correct arguments

In [ ]:
# TODO: Calculate the three performance metrics for the neural network and compare them to the linear regression and random forest models

In [ ]:
# TODO Create the final parity plot, comparing all three models

### Model interpretation

A linear regression being simply a linear equation fitted to a set of training data, its coefficients can be analysed and can, if the model is trusthworthy, allow us to estimate the importance of each descriptor for the prediction task. This point will be discussed in the questions below, but let's start by plotting the values of the coefficients found by our linear regression for each descriptors we gave as input.

In [ ]:
# Plot the coefficient importance
plt.figure(figsize=(8, 5))
plt.bar(range(X.shape[1]), lin_reg.coef_, tick_label=[features_list[i] for i in range(X.shape[1])])
plt.xticks(rotation=90)
plt.xlabel("Features")
plt.ylabel("Coefficient value")
plt.title("Linear regression coefficients")
plt.show()

## Questions



1. In this exercise we have trained and evaluated three different models, a linear regression, a random forest and a neural network. ML models have different number of parameters, i.e. internal variables the model learns and optimises during the training to map the input features to the target value. While the random forest has multiple thousand parameters (nodes) and the neural network has close to 300 000 trainable parameters, estimate how many trainable parameters the linear regression has.

2. Rank the models according to their complexity. Can you guess what is the danger with more complex models and little training data?

3. We calculated scoring metrics to compare the predictive performance of the models. Can you confidently say which method performed the best in our case?

4. Looking at the coefficients in the linear regression, we could draw conclusions on the importance of different descriptors for the prediction of absorption wavelengths. How much would you trust these coefficients and the interpretation of their importance?
